In [1]:
import csv, io
import json
from bs4 import BeautifulSoup
import codecs, prettytable
import requests
import codecs
from selenium.webdriver import Chrome


header_data={
		"User-Agent" : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.138 Safari/537.36",
		"Accept-Language": "zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7",
		"referer" : "https://forum.gamer.com.tw/B.php?bsn=60559&subbsn=7"
		}

In [2]:
# 取得所有標題的 snA

# --------subbsn----------
Apple = "7" 
p_Apple = 18

Sony = "13" 
p_Sony = 9

Samsung = "12" 
p_Samsung = 10

XiaoMi = "11" 
p_Mi = 6

Google = "22" 
p_Google = 2

# 需設定參數
classNow = Apple 
page_count = p_Apple 
def get_snA(page_count, classNow):
    snA_list = []
    for page in range(1, page_count+1):
        r1 = requests.get(
                f"https://forum.gamer.com.tw/B.php?bsn=60559&subbsn={classNow}",
                headers=header_data,
                params={
                    "page":str(page)
                    }
            )
        b1=BeautifulSoup(r1.text, "html.parser")
        a1 = b1.find_all("td",{"class":"b-list__summary"})

        for a2 in a1:
            snA = a2.find("a")
            # print(snA.attrs["name"])
            snA_list.append(snA.attrs["name"])           
#     snA_list.remove("48476")      
    return snA_list

# snA_list = get_snA(page_count, classNow)
# print(snA_list)
# print(len(snA_list))



['50305', '50269', '50277', '50272', '50257', '50236', '50231', '50226', '50109', '49906', '50083', '50073', '50076', '50066', '50064', '50038', '50021', '50023', '49991', '49985', '49962', '49938', '49898', '49896', '49894', '49886', '49874', '49857', '49698', '49819', '49795', '49788', '49785', '49721', '49724', '49702', '49526', '49688', '49656', '49679', '49661', '49652', '49640', '49600', '49581', '49539', '49512', '49493', '49468', '49407', '49398', '49394', '49373', '49287', '49271', '49264', '49262', '49253', '49084', '49221', '49176', '49209', '49186', '49120', '49117', '49087', '49102', '49094', '49063', '49059', '48971', '48884', '48860', '48851', '48823', '48794', '48765', '48777', '48770', '48768', '48759', '48734', '48711', '48699', '48634', '48598', '48613', '48583', '48590', '48571', '48503', '48504', '48485', '48484', '48479', '48469', '48438', '48432', '48404', '48365', '48320', '48296', '48297', '48182', '48166', '48164', '48087', '48077', '46876', '42799', '47418', 

In [3]:
# snA_list=[50305]
# snA_list.append("49698")
# print(snA_list)
# snA_list.index("42749")
# a = snA_list.index("42749")
# snA_list = snA_list[a:]
# snA_list

In [4]:
def get_floor(bs):
    floor_list=[]
    floor = bs.find_all("p",{"class":"BH-pagebtnA"})
    
    for f in floor:
#         print(f.text[-1])
        floor_list.append(f.text[-1])
#   print(floor_list)
    return floor_list

In [5]:
def get_page_parser(snA, classNow):
    r = requests.get(f"https://forum.gamer.com.tw/C.php?bsn=60559&snA={snA}&subbsn={classNow}",
                     headers=header_data,
                     params={
                     }
                     )
    r.encoding = "utf-8"
    bs = BeautifulSoup(r.text, "html.parser")
    return bs

In [6]:
def get_commentId(snA, floor, classNow):
    r2 = requests.get(
                f"https://forum.gamer.com.tw/C.php?bsn=60559&snA={snA}&page={floor}&subbsn={classNow}",
                headers=header_data
            )
#   print(range(1, int(page_list[0]) + 1))
    print("正在處理第幾頁:", floor)
    commentId_list=[]
    b2=BeautifulSoup(r2.text, "html.parser")
    a1 = b2.find_all("textarea")
    for a in a1:
        commentId_list.append(a.attrs["data-snb"])
#     print(commentId_list)
    return commentId_list


In [7]:
import time

# 取得該snA展開留言編號
with open("0603.csv", "a", encoding="utf-8", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["標題", "內容", "使用者評論"])

for snA in snA_list:
    
    data = []
    
    bs = get_page_parser(snA, classNow)
    floor_list = get_floor(bs)

    driver = Chrome("./chromedriver_V83")
    for floor in range(1, int(floor_list[0]) + 1):
        
        commentId_list = get_commentId(snA, floor, classNow)
        driver.get(f"https://forum.gamer.com.tw/C.php?bsn=60559&snA={snA}&page={floor}&subbsn={classNow}")

        for id_ in commentId_list:      
            try:
                driver.find_element_by_id("showoldCommend_"+ id_).click()
            except:
                continue   
#     ------------------------------------------------------------------------------------- 
        user_name = driver.find_elements_by_class_name("reply-content__user")
        comment = driver.find_elements_by_class_name("reply-content__article")
        content = driver.find_elements_by_class_name("c-article__content")
        title = driver.find_elements_by_class_name("c-post__header__title ")
        user_name_list = []
        title_list = []
        content_list = []
        comment_list = []
        result = dict()
        
        for name in user_name:
            user_name_list.append(name.text)

        user_name_list2 = list(set(user_name_list))
        print(user_name_list2)
        for til in title:
            title_list.append(til.text)
            print(til.text)
        for con in content:
            content_list.append(con.text)
            print(con.text)   
        for com in comment:
            s = com.text
            for n in user_name_list2:
                if n in s:
                    s = s.replace(n, "")
            comment_list.append(s)
            print(s)  
        
        result["title"] = title_list
        result["content"] = content_list
        result["comment"] = comment_list
        data.append(result)
        
        with open("0603.csv", "a", encoding="utf-8", newline="") as f:
            writer = csv.writer(f)
            for item in data:
                writer.writerow([item['title'], item['content'], item['comment']])
                     
    driver.close() 
        

正在處理第幾頁: 1
['151313']
['小鸚', '結月葵我婆', '能看見的隱形人', '夏日', '天璇星河']
【問題】iPhone se2問題
不好意思打擾下，請問有人買了iPhone se2出現用safari時鍵盤失靈問題嗎？或是在某些程式裡點擊鏈接跳出Safari遊覽器時也失靈的問題嗎？謝謝QQ
是鍵盤按了沒反應?!
 對呀... 內置的輸入法，或商店下載的輸入法都試過了有一樣的問題
有試過別的瀏覽器? 我目前還沒遇到 我的版本是13.4.1
去設定那重置試試看
好哦 我試試看
謝謝你們..
我的沒問題 你大概機王 退貨
欸鳳有出現過鍵盤失靈的問題，你如果放置十分鐘到十五分鐘，它就正常的話。估計就是一樣狀況
正在處理第幾頁: 1
['151176', '151180', '151197', '151200', '151214', '151216', '151228', '151234', '151241', '151248', '151253', '151269', '151277']
['韭菜盒子', 'J', 'Smart', '風', '荒武陽一', '煎餅好ㄘ', '櫻京inin', '深水蒼藍', '比歐雷杰', 'shakaka', 'MR.KIM', '多蒙卡修', '国木田小丸', '伊海路·魯洛歇', '疏筋活血', '酸言酸語雙馬尾', 'zero', 'KAI', '45kg肥宅', '我對普通人沒興趣', 'Lza', 'a112233', '壞米亞', '無法抹滅の胖次', '殘月', 'iPhoneＸ的瀏海', '結月葵我婆', 'padra', '浩呆', '搞笑', 'weison', '人性毀滅', '蔬', '棉姆咪雪兒', '看故事的人', 'ra1xp', '★御惠明希♡', '阿瘋', '千妞V', '阿杰', '神秘怪客', '你是認真的嗎？', 'Mshinaoa', '樹上的Apple不能吃', 'sunshare', '我是來亂的', '偃柳', 'X洨妹', '通知達人', '小胖', '海鮮好吃', 'Plus', '佐和子', '惡魔華子', 'KID', '黑', '淺水', '口瓜口瓜~', '獵神人']
【問題】沒有遊戲需求，iPHONE 6 Plus還堪用嗎?
如題，有個親戚要脫手她

['★御惠明希♡', '通知達人', 'KAI', 'KID', '翔', '結月葵我婆', 'weison', '搞笑', '黑子舞想']
【問題】買11 或 xs
哈囉各位晚上好
最近有想換新手機的慾望 目前6s巡航真的很不夠力加上玩遊戲超耗電
目前在考慮這兩隻，但看到許多人都說12快出了現在先別買 我該繼續等下去還是直接買這兩隻其中一隻... 小弟預算大概25左右
換個電池再戰十年
iPhone沒那麼不耐用吧
看你要續航比較持久
還是比較高解析度的螢幕

自己之前是選了11
省電有感 出門不用一直帶尿袋就是爽
我自己用11，如果手機真的瞠不下去就換11吧。

一來比較新，二來電池真的很有感。我前一台是8+，大螢幕加上高解析很爽，但可悲電池換來的就是照三餐充電。

11的解析度雖然有點悲劇，但電池強太多了，續航很有感，至少不用時常擔心電量
整體而言11會好很多了，如果你沒有很要求的話。
是我的話會選11，覺得實用性比較高。
真心建議11
 不太想等12ㄌ 應該會直接入手11
任性
都有結果了也不需要問了吧，11也夠你用3年以上了了
 就是任性( ´▽｀)
 也就單純想看看大家的想法而已@@
11除了大隻厚重、邊框粗、畫質較低外其他全贏Xs啊
重手遊當然11
 有拿過同學的xs跟11比較，XS畫面真的很細膩，手感好 但我又考慮到效能的差距....有看外國YOUTUBER說如果換11可以比換XS多撐一年?
 其實也沒很重，就偶爾開跑跑起來玩而已，手游真的沒幾款好玩的qq 都好速食
 重視性能就買新不買舊了，如果日常使用我看a12和a13都能用個四五年，11多撐一年可能是指ios更新多享受一個版本？
 好 剛剛去查了一下被紫色燒到 超美的啦 考慮去地標買
沒比較沒傷害
11吧剛從安卓換蘋果 一整個舒服
幾個月前已經換了電池 但我有配合AIRPODS使用 導致續行很差QQ
iPhone沒那麼不耐用，那是APP優化差，換個電池再戰十年吧XD
換電池後還這樣 那就換機吧XD
不太想等12了XD 11真的好香
i11 pro 也不差 純粹兩代的差異
好的 目前也是考慮11 目前爬文XS解析度高真的耗電...續航真的悲劇
正在處理第幾頁: 1
['151133']
['沒禮貌的雞雞']
【問題】IPHONE apple id 已鎖定 求解!!
請店家找回密碼但是家人拿回來後就被

正在處理第幾頁: 1
['149824', '150630']
['Xuin', '蔬', 'Wei', '神奇燒酒螺']
【問題】iPhone XR 手機問題導致網路速度變慢
大家好是這樣的~

我的 iPhone XR 已經更新到 IOS 13.4.1 一段時間了, 網路本來都非常順暢 (中華電信)

一直到兩天前網速突然超慢,變只有 1~9Mbps

也變超耗電 , 本以為是SIM卡問題 , 換到iPhone 7去測網速 , 竟然有70Mbps ...


看來是手機問題了 , 大家知道可能是哪裡有問題嗎?    硬體軟體方面的話 ....
※ 引述《Xinhua807 (Wei)》之銘言
> 大家好是這樣的~
> 我的 iPhone XR 已經更新到 IOS 13.4.1 一段時間了, 網路本來都非常順暢 (中華電信)
> 一直到兩天前網速突然超慢,變只有 1~9Mbps
> 也變超耗電 , 本以為是SIM卡問題 , 換到iPhone 7去測網速 , 竟然有70Mbps ...
> 看來是手機問題了 , 大家知道可能是哪裡有問題嗎?    硬體軟體方面的話 ....
> 在新視窗開啟圖片

啊 終於撐到 IOS 13.5 更新了
Xuin 果然變快了

但之前IOS 13.4.1也用一段時間好好的 , 真是奇怪 ...
分別重開機->備份重置試試看,啊訊號只有一格當然慢
謝謝 , 有重開機跟重置網路設定過了 , 4G滿格時 也只有9Mbps QQ
看有沒有辦法借到其它張卡來測試,萬一還是一樣,那你就要考慮是硬體問題了
對 還是一樣 嗚嗚 其他功能都很順也很快的說
intel基带的问题吧，还有ios13.4.1bug也不少，不排除是系统原因，期待下一个版本的推送更新
天線沒扣好（×
正在處理第幾頁: 1
['150520', '150522', '150553']
['阿杰', '蒼月', '煎餅好ㄘ', '龜龜', 'mumi5566', '佇立在翡翠海的少女', '結月葵我婆', '天堂粉絲', '珈琲しば', '樹上的Apple不能吃', 'ლ(́◕◞౪◟◕‵ლ)', 'Scott', '阿綠 ♉', '幻滅星旋', '緣起仙子']
【問題】從三星平板S2 跳到IPAD PRO 2020
使用了快6年的三星平板S2，被更新害慘後不勘使用，本來是沒打算入手

正在處理第幾頁: 1
['150049']
['空氣感我愛騎爆改山豬', '(´●ω●)ﾉ(._.`)', 'Scott', 'Xuin', '蔬']
【問題】怎麼刪除apple id?
我把舊I6+手機給我爸用 結果他忘記密碼
回復模式之後要登入apple id
帳號密碼都正確但要解鎖帳號
驗證顯示的信箱他說不是他的 改填生日也失敗
這下好了 沒人能登入這台Iphone 也解綁不了這台手機
有沒有辦法可以刪掉這個apple id改登別的帳號呢？
無法登出
 真的哦...
gg了
如果有方式購買的包裝盒和收據的話可以去蘋果官方解鎖，否則就是塊板磚了
發票跟包裝盒拿回去找蘋果，不然完全沒救
包裝盒還行 發票的話太久了 難找
正在處理第幾頁: 1
['149968']
['沙比', '凜崎羽夜', 'Scott']
【問題】買錯禮品卡
那個，有人試過apple store禮品卡(日本)可以在台灣用嗎 買配件那個。     我要買點數的買錯了。導致我現在卡了個10000日圓的禮品卡在錢包裡


還是只能等我有生之年去日本一趟花掉它？
不能,禮品卡等是點數卡那類鎖區的東西
 所以我只能等以後有機會飛日本去他們的直營門市把他花掉了哦？
如果有日帳玩遊戲課金也可用
 不是。 我買到的是apple store 配件商店的卡。不是app商店的也可以買點數？
store卡那種一樣可以開日帳,在日帳官網買東西
只是運費會蝕本
你是不是原本想買itunes & app store禮品卡 結果買到配件卡
好的。感謝
 沒錯的啦.....
正在處理第幾頁: 1
['149793', '149817']
['ra1xp', '工作紓壓', 'Fang', '甜心', '今天是特例歐(o^^o)', 'beyerdynamic))))', '一鐵', '▬ι═════ﺤ', '鬆餅&隼ᕕ( ᐛ )ᕗ']
【問題】iPhone11耗電問題
上個月從大8換到11，但剛拿沒幾天就感覺耗電怪怪的 本來以為是錯覺 但最近好像耗的越來越快⋯

昨晚11點充飽拔掉電源 就一直到早上6點起床才打開（睡覺有開月亮）
然後7點開始上班到只有碰過一次手機 現在電量剩68%
請問這是正常的嗎ಥ_ಥ
有開飛航正常不太可能這麼耗電
這是我的
不太正常吧
用幾天而已不準吧…起碼一兩個星期
怪怪的，5/7凌晨3點到6點前都沒使

正在處理第幾頁: 1
['149528']
['しらかわみやこ', '黑', '佇立在翡翠海的少女', 'd(`･∀･)b', 'Ryu Beckham', '沙比']
【問題】Ipad定位問題
我目前使用的是Ipad air3代 ，平常都是連手機網路，但是定位卻一直都有問題
開怪物彈珠連線也都是無法定位

定位明明都是開著，也有授權了
想問問有沒有人跟我有一樣問題的，或者是有解決方法的
LTE版才有GPS，wifi版沒有
有wifi定位，這樣不正常
 如果正常是怎麼樣
我的都能正常定位，偶爾略飄，市區內都挺準的
 還是我住的地方比較鄉下的關係
 有可能，wifi定位主要依據你住家附近的wifi來推斷你所在的位置；下載Google地圖定位看看
 欸都，我試過ㄌ好像也沒辦法
wifi版可以定位 試試重置定位與隱私權或者去都市測測看
鄉下地區可能沒有定位用的SSID可參考，所以無法用Wi-Fi定位。
正在處理第幾頁: 1
['149456']
['櫻京inin', 'wkry', '初四了阿北']
【問題】關於i8plus切換app
請問iphone8 plus有人遇到按兩下home鍵沒反應的嗎？但是回主畫面又正常了
代表你該換最新的SE了
有 而且最近越來越容易發生 有一次還要重開機 暈
 哈哈！不習慣小螢幕
正在處理第幾頁: 1
['149428']
['蔬', '湯姆士小火車', 'Scott']
【問題】這個是啥問題? 使用google APP首頁很滑動很卡
我是拿I7 +

不知到哪時候更新完後開始這個問題,之前正常打開估狗收尋下面會有一些新聞的報導或遊戲

之前滑動時後很順暢,最近滑動看那些新聞字卡很不順暢,還沒點進去看(點去進看滑動是很順暢

使用估狗收尋完其他網站滑動也是很順暢

很奇怪唯獨估狗首頁滑動很卡  試過重新安裝一開始有感 後面又開始卡 有關掉所有背景程式
手機容量還有17GB沒用

改用6S卻沒這問題
你是說Google.com.tw?
safari
甚麼意思 我是抓那個app下來用的
用不習慣
火狐、brave、opera都比谷歌好用還快 app store建議到設定關自動更新 應用原本好好的可以用 幹嘛手殘按更新 更新是手機變慢變卡的元凶
谷歌那些資訊卡很爛 感覺都在偷分析個人喜好跟參雜廣告 看到就覺得噁心
更新不一定會變慢變卡，分析你可

...

正在處理第幾頁: 1
['121609']
[]
【情報】蘋果2018秋季發表會 iPhone XR、XS、Max展示機影片
看好看準下好離手
Youtube 小齒輪 可自動翻譯 繁體中文



Apple Watch Series 4
正在處理第幾頁: 1
['121578']
['狂一', '笑點很低的福音矮冬瓜', '雪空', '骨頭', '鋁罐頭', '給予世界痛苦']
【問題】XS MAX 256 512讀取速度
以過往的經驗

高容量會在讀取速度上勝過低容量

但往往看到的文章都是32或64跟128以上的做比較

這次256跟512倒是第一次

不知道各位果粉如果不考慮其他因素
256跟512素上差異會很大嗎

單看容量的話，容量越大代表 Flash 顆粒數越多，I/O 數越多，代表可以同時寫入的資料量越大，就好比電影院散場時，出口的數量越多，就可以越快疏散完人潮。
以上是我爬文看到的

我是個想換APPLE的新用戶
來問大家這個菜菜的問題 希望別見怪
沒差，要用蘋果請不要在意任何有關"數字"的東西
速度差不了多少 容量差了很多
差了4倍... 可是用不到
4倍是說64跟256嗎
喔沒看到你說256 我是用128跟512比
理論上有差 但幾乎感覺不出來
恩恩的確測試機器有差 但用戶無法感受到差異
蘋果的讀寫一直都是頂尖水準，所以其實根本沒差
正在處理第幾頁: 1
['121555']
['偽乳酸飲料', '赤龍控', 'BLUEDOG', 'iPhone 8 Plus', 'kit', 'Sleepypanda']
【情報】2018全新iPhone發佈會懶人包！！！
熬夜把這次的Apple iPhone Xs、Xs Max 以及 iPhone XR的發布會給看完了，自己剪接了一下發布會的影片!!
並做了字幕讓大家能快速知道這次三隻手機的重點。
希望大家看了會喜歡!!!
感謝大大製作懶人包!
謝謝整理 蘋果又帶來創新跟未來
感謝，小米9外觀發佈會
懶人包就是留錢買GTX2080
RTX
顯示卡跟手機是不同的東西吧....
正在處理第幾頁: 1
['121463', '121466']
['啪答啪答碰', '小惡魔', 'Wa_Si_Dv', '黑羽紀之', '紅茶屋']
【問題】廣告
用手機看YouTube 一堆廣告
有神人可以推薦免費阻擋廣告的app嗎？

拿蘋果價錢在那說嘴 啊三星更貴都裝作沒看到還是三星是雜牌都被你們忽略了
連世界第二的華為都能忽略 三星當然也是啦
我在大陸 現在連低收入戶 月領3000 4000RMB的都人手一隻蘋果 華為真的不知道是銷到哪邊去 感覺跟跑分事件一樣造假....
推樓上
毛蟲幫
iPhone在中國市佔率才第五
無產階級幾乎拿華為小米
市佔率比較低但是蘋果賺得錢是華為的好幾倍呵呵呵呵呵呵
拿什麼手機跟有沒有錢是兩回事，路上一堆雙逼有幾台是真的靠自己能力賺的？
兩個不同系統啊 只有三爽被華為幹掉可以期待吧?畢竟大陸人口超多啊
有錢的支那人，也是選蘋果、三星旗艦，不會去選華為吧…
三爽拉雞獵戶座就別提了吧
雖然不喜歡蘋果，不過去年瀏海屏跟全屏手勢也能叫了無新意？
不可能啦 拉下神壇除非果粉都覺得不行了才有可能
蘋果有一連串的商品 反觀安卓有嗎?所以用了蘋果的人不太可能會放棄蘋果的
很多人都只有用iphone吧
瀏海屏那是迫於成本，不是因為創意吧？再看看i6~i8，還真沒啥新意。iX那算很有創意的話…，all right
有手機就會買筆電 就會買平板 就會買桌機的
那是信仰強烈到不行的果粉，不是一般果粉…
只有用iPhone也別小覷跳槽的麻煩,手機系統是說跳就跳,但你在Apple Store上的購買的東西也只能在iOS上用
喜歡iPhone、ipad但覺得iMac不好用 可能文書處理比較方便吧 聽別人講用一次就會愛上了==
IMAC因為操作上跟微軟不一樣 所以有比較便利點
安卓陣營有一系列的商品我目前只知道小米比較多，手機和他的智能家庭設備可以做連動，其他安卓廠商在設備連動就非常少
信基督教的不太會改信佛教吧
我是覺得，除非真的資金很充足加上信仰很深，不然不太可能這樣買一堆週邊
我姐買手機 但沒買桌機 沒買筆電 沒買ipad 她是不是假果粉
Re_Dash 應該說有錢人大部分會挑的牌子就那幾樣XDD 你這樣說有點偏哈哈
自從賈柏斯掛了後，蘋果就不曾讓人有驚艷感了......至於操作簡易性還是看老人領悟力，要說卓越真的未必
 要做設備連動就要弄到生態圈那樣各類產品都有(或合作)，這種除非有在認真砸錢經營不然我認為一般廠商做不來
一般只要能做到適配支援就已經很強了XD
照蘋果現在的狀態被拉下只是"時間"，畢竟這幾間中國大廠在好幾年前就已經做好新技術研發人才儲備跟專利應用項目研發的比重增加了很多


每朝健康爽健美茶 就跟假果粉在那邊越獄說蘋果好用一樣噁心，我以前就是這麼看那些自稱「果粉」的
我之前看到三星廣告嘲諷蘋果，我只想說三星廣告拍的好，真棒，瀏海醜死了
安卓要刷不都是因為除了旗艦機跟少數機型外廠商基本上都賣後不理 系統版本更新不給就算了 連安全性更新也沒有 就算你發再多的抱怨文 也不會憑空跑出更新 那怎麼辦 自立自強自己刷咩
總不能說 叫大家都買旗艦吧 預算不是大家都那麼多
磷 所以我看到刷機的米粉說買蘋果的都盤子，我是笑笑不說話啦，因為人家售後服務，刷機米粉怎看的到呢？
三星也是啦，之所以賣這麼貴，還不是有些應用真的好用，一堆人只單看規格說三爽垃圾，我也覺得很好笑
對於沒那麼多錢買旗艦 但又想把性能發揮極致的人來說 刷機就是唯一的路
看得到啊 小米刷機沒破保 照樣可以走售後
每朝健康爽健美茶 所以買三星蘋果的人就得被罵？三星蘋果哪裡惹到那群人了？
三星刷機也能得到更好的使用體驗 比較少人刷的原因就是因為破保 之前就拿過ote4刷ote7移植過來的ROM...
因為在差不多的硬體條件下 兩個品牌都能刷 但價錢差了很多 這就是被嘴的原因
每朝健康爽健美茶 所以你拿ote8刷機了沒？三星那些用心你有看到？別把刷機當作手機好壞的標準
有些人眼睛有問題 看不到價格高的項目多出的價值在哪...
我ote8有在用Knox用得好好的 沒事去刷機讓這個功能失效做什麼...
磷 恩，所以開法拉利藍寶堅尼的人就要被嘴，因為車子能開就好
每朝健康爽健美茶 這就是三星在ote系列上的努力啊，你不想刷不就代表三星成功了？
通常會買到那麼貴的東西 應該是不會去理別人說什麼的。。 他連這麼貴的都買的起 便宜的當然也能買啊 有些人的邏輯就不好啊...
這不代表說我完全不想刷ote8 有想過買下一台ote之後 在看XDA上面的刷機資源有沒有吸引到我再開刷...
把新ote的有的東西刷進舊ote裡面 倒是常常看到喜歡ote系的人這樣刷...
每朝健康爽健美茶 因為那是對應Spen增加新東西，什麼文字懸浮翻譯，三星就是靠這個走出安卓
如果照你這樣分 那這樣搞的人就不是ote粉了...
每個品牌都有人把新機的新功能移植刷到舊機上 我倒認為這樣刷仍然算那個品牌的粉絲...
蠻想知道oppo多出的價值在哪....
每朝健康爽健美茶 三星系統更新慢，所以我不選三星很久了，我最早接觸三星是它的平板
照

被封面嚇到
期待惡搞頭像
其實face id 直覺多了
是嗎?可是我並不這麼認為,指紋鎖不要你手濕,或者有污垢,幾乎都是碰到就立即解鎖,而face要等個1-2秒,讓它完成辨識,辨識不出來就要輸入密碼,這反而會更不方便一些些
我個人用的想法是覺得，face辨識度高於指紋非常多，指紋越用，辨識度會越來越不靈敏
但face速度的確是輸給指紋，可是如果把手放到指紋感應觸，再換回正常用手機的姿勢的話，我認為速度是差不多的
並行會是更好選擇，不過價錢就...
我都是點開螢幕直接滑他就會跟著解鎖了
正在處理第幾頁: 1
['102627', '102628', '102669', '103603']
['上官雜魚', '墮天使_厭禁', '真冬最高', 'Yeeee', 'Houl', '何翊', '睿又哥', '小雞勇猛', 'mongogoro', '°BAHA柏宏〞★', '彥仔', '愛著雛田的忍者', 'Andy', '湘雪', '哈比老司機', '♍埃~低~打補溜~~打喵', '鄭曉麥(進修中勿打擾)', '滄海', '壞熊熊', '黑', 'Stay4Real', '彈珠 滾！', '亞洲提督', 'eggyboy', 'QooQPOWER', '大紅', 'Helix Tung']
【心得】iPhone X使用心得和部分功能雜談(加上跑分)
前言：

iPhone5用了五年終於撐不住了
最近一直在使用實驗室的HTC one當備用機
但是用Macbook配上安卓時在不是什麼好選擇...畢竟蘋果就愛搞全家桶
(雖然我也有專門玩遊戲用的PC 但非玩遊戲正事都用Macbook 大半時間都在研究...)
有時真的會讓你覺得想摔手機
當時索性就決定X應該會入手了

購買價格：

我購買是買網路門市的綁約
大致上iPhone5舊機底1300(外觀磨損被扣200 不過檢查機子的人說這也保養太好 笑)
加上學生專案抵6000(月費589x30) 大概27000多買到64GB的iPhoneX 黑色

硬體方面

CPU＆GPU：
這次iPhoneX的核心A11(4+2 core CPU)
有可以做ML 自家Design的GPU
以這過高的硬體基本上沒啥遊戲可以真的使用到
至於跑分到處都是了
可以提一下的是iPhoneX的A11有稍微提升過 是2.23GHz 相較於8的是2.06GHz

※ 引述《bmwamg888 (bmwamg)》之銘言
雖然這樣說可能太嗆了，但是

可以請你不要秀下限嗎

有玩音樂的都知道

鍍金鍍銀雖然都可以抗氧化

但是鍍銀的傳導率最好

可以讓歌曲、聲音的表現比較好

這也就是為什麼有些對音樂很執著的人

會去把音樂播放的相關器材的線材從銅線改成銀線的原因

銅線的傳導率好，但是銀線更好

那麼，這個轉接頭是用來聽音樂的

不是用來傳輸資料的

一樣鍍金鍍銀都能抗氧化

當然要選對音樂品質更好，傳導率好的鍍銀啊

在那邊喊什麼costdown的

真的有點丟臉

只是顯現你的知識淺薄而已
個人認為覺得音質變好(或者說是有變化)應該是dac用得不一樣的關係，和是不是用3.5mm介面關係不大
據我所知，lightning介面最終輸出的仍然是類比訊號，差別只在於dac是在手機內部還是外部
※ 引述《abigolight (Abigo)》之銘言
> 雖然這樣說可能太嗆了，但是
> 可以請你不要秀下限嗎
> 有玩音樂的都知道
> 鍍金鍍銀雖然都可以抗氧化
> 但是鍍銀的傳導率最好
> 可以讓歌曲、聲音的表現比較好
> 這也就是為什麼有些對音樂很執著的人
> 會去把音樂播放的相關器材的線材從銅線改成銀線的原因
> 銅線的傳導率好，但是銀線更好
> 那麼，這個轉接頭是用來聽音樂的
> 不是用來傳輸資料的
> 一樣鍍金鍍銀都能抗氧化
> 當然要選對音樂品質更好，傳導率好的鍍銀啊
> 在那邊喊什麼costdown的
> 真的有點丟臉
> 只是顯現你的知識淺薄而已

你玩音樂甘我啥事!   但我也真的沒在玩音樂!
我是以製造業的角度來看產品
電鍍金(非化金),在業界插拔測試,本來就是最耐磨的!
(更高階的產品,是全面電鍍金,也就是線路有鍍金)
好比說USB3.0高速傳輸(高頻), 此類相關的產品

數位資料從手機傳出, 然後在外部接頭裡的晶片轉換成類比 , 這跟傳導率有啥關係?
傳0101010101這種數位資料要扯到傳導率?
那你乾脆要不要改光纖的接頭? 用光來傳資料
感覺兩邊都有理....lightning端接收的是數位輸入，理論上只要能正確傳導資料就好，鍍銀鐵定不會升級音效。3.5那端是類比輸出，鍍銀或許就會有幫助了。

http://www.techbang.com/posts/46271-violence-and-dismantl